In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from datetime import datetime

df = pd.read_csv('Final_Data.csv')

df['Time'] = pd.to_datetime(df['Time'], format='%d-%m-%Y %H:%M')

df['Needs_Service'] = df['Probability of Failure'].map({'High': 1, 'Medium': 0, 'Low': 0})

label_encoders = {}
for column in ['Machine', 'Component', 'Parameter']:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

X = df[['Machine', 'Component', 'Parameter', 'Value']]
y = df['Needs_Service']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

service_model = RandomForestClassifier(n_estimators=100, random_state=42)
service_model.fit(X_train, y_train)

y_pred = service_model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

def predict_service_need_and_days_left(machine, component, parameter, value, current_time):
    # Encode the inputs
    machine_encoded = label_encoders['Machine'].transform([machine])[0]
    component_encoded = label_encoders['Component'].transform([component])[0]
    parameter_encoded = label_encoders['Parameter'].transform([parameter])[0]

    input_data = [[machine_encoded, component_encoded, parameter_encoded, value]]
    
    service_prediction = service_model.predict(input_data)
    
    if service_prediction[0] == 1:
        return "Service Needed", 0  
    
    else:
        similar_cases = df[(df['Machine'] == machine_encoded) & 
                           (df['Component'] == component_encoded) & 
                           (df['Parameter'] == parameter_encoded) & 
                           (df['Value'] >= value)]
        
        if not similar_cases.empty:
            similar_cases['Days_Until_Service'] = (similar_cases['Time'] - current_time).dt.days
            average_days_left = similar_cases['Days_Until_Service'].mean()
            return "No Service Needed", max(0, round(average_days_left))
        else:
            return "No Service Needed", "Insufficient data to estimate days left"




Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       176
           1       1.00      1.00      1.00       124

    accuracy                           1.00       300
   macro avg       1.00      1.00      1.00       300
weighted avg       1.00      1.00      1.00       300



In [ ]:
def interactive_service_prediction():
    machine = input("Enter machine name: ")
    component = input("Enter component name: ")
    parameter = input("Enter parameter name: ")
    value = float(input("Enter value: "))
    current_time_str = input("Enter current time (format: dd-mm-yyyy hh:mm): ")
    
    current_time = pd.to_datetime(current_time_str, format='%d-%m-%Y %H:%M')
    
    service_status = predict_service_need_and_days_left(machine, component, parameter, value, current_time)
    print(f"Service Status: {service_status}")

interactive_service_prediction()